In [31]:
import os
os.environ['PATH']

'C:\\Users\\Ahmad Ardi\\Anaconda3\\DLLs;C:\\Users\\Ahmad Ardi\\Anaconda3;C:\\Users\\Ahmad Ardi\\Anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\Ahmad Ardi\\Anaconda3\\Library\\usr\\bin;C:\\Users\\Ahmad Ardi\\Anaconda3\\Library\\bin;C:\\Users\\Ahmad Ardi\\Anaconda3\\Scripts;C:\\Users\\Ahmad Ardi\\Anaconda3\\bin;C:\\Users\\Ahmad Ardi\\Anaconda3;C:\\Users\\Ahmad Ardi\\Anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\Ahmad Ardi\\Anaconda3\\Library\\usr\\bin;C:\\Users\\Ahmad Ardi\\Anaconda3\\Library\\bin;C:\\Users\\Ahmad Ardi\\Anaconda3\\Scripts;C:\\Users\\Ahmad Ardi\\Anaconda3\\Library\\bin;C:\\Program Files (x86)\\Common Files\\Oracle\\Java\\javapath;C:\\Program Files (x86)\\Intel\\iCLS Client\\;C:\\Program Files\\Intel\\iCLS Client\\;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\;C:\\Program Files (x86)\\Intel\\Intel(R) Management Engine Components\\DAL;C:\\Program Files\\Intel\\Intel(R) Management Engine Components\\DAL;C:\\Prog

In [32]:
from pandas import DataFrame
import pandas as pd
import numpy as np

In [33]:
from jsonapi_client import Session, Filter
import pycurl
import html

API_BASE = "https://www.ebi.ac.uk/metagenomics/api/latest/"

In [34]:
import os, sys
from tqdm import tqdm
import ipywidgets
from time import sleep
import glob

In [35]:
def get_metadata(metadata,key):
    for m in metadata : 
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}". format(value=value,unit=unit)
        return None
    
def get_study(term,lineage,biome,filename):
    if not os.path.isfile(filename):
        with open(filename, 'wb') as f:
            c = pycurl.Curl()
            c.setopt(c.URL, 'https://www.ebi.ac.uk/metagenomics/api/v1/studies?lineage=root%3AHost-associated%3AAlgae&ordering=-last_update&format=csv')
            c.setopt(c.WRITEDATA, f)
            c.perform()
            c.close()
    return filename

def get_analysis_result(run, extension):
    API_BASE_RUN = 'https://www.ebi.ac.uk/metagenomics/api/latest/runs'
    with Session(API_BASE_RUN) as s:
        study = s.get(run,'analysis').resource
        for i in study.downloads:
            if extension in i.file_format['name']:
                link = i.url
    return link

def random_sampling(dataframe, amount):
    df_random = DataFrame(columns=('secondary_accession','url','samples_count','bioproject','accession','biomes','centre_name','last_update', 'study_abstract','is_public','study_name','data_origination'))
    df_random.index.name = 'No'
    a = 0
    while a < amount:
        i = np.random.choice(dataframe.index.values, 1)
        container = df_random.loc[:, 'secondary_accession']
        if not container.isin([dataframe.loc[i[0], 'secondary_accession']]).any():
            df_random.loc[i[0]] = [dataframe.loc[i[0], 'secondary_accession'], \
                                   dataframe.loc[i[0], 'url'], \
                                   dataframe.loc[i[0], 'samples_count'], \
                                   dataframe.loc[i[0], 'bioproject'], \
                                   dataframe.loc[i[0], 'accession'], \
                                   dataframe.loc[i[0], 'biomes'], \
                                   dataframe.loc[i[0], 'centre_name'], \
                                   dataframe.loc[i[0], 'last_update'], \
                                   dataframe.loc[i[0], 'study_abstract'], \
                                   dataframe.loc[i[0], 'is_public'], \
                                   dataframe.loc[i[0], 'study_name'], \
                                   dataframe.loc[i[0], 'data_origination']\
                                  ]
            a = a + 1
    return df_random

In [36]:
#Search Study
term = 'Algae'
lineage = 'Host-associated'
biome = 'root'
file_study = '01_Study_'+term+'+'+biome+'+'+lineage+'_raw.csv'

In [37]:
#Download study information
get_study(term, biome, lineage, file_study)

'01_Study_Algae+root+Host-associated_raw.csv'

In [38]:
#Load Study information
df1 = pd.read_csv(file_study)
#df1

In [39]:
#Select relevant information
df_study = df1[['secondary_accession','url','samples_count','bioproject','accession','biomes','centre_name','last_update', 'study_abstract','is_public','study_name','data_origination']]
#df_study

In [40]:
#Add information on biome
df2 = DataFrame(columns=("Biome","Lineage","Latitude","Longitude","Publication"))
#df2.index.name = 'No'

for i in tqdm(range(len(df_study))):
    with Session(API_BASE) as s:
        std = s.get('studies', df_study.loc[i, "secondary_accession"]).resource
        for a in std.biomes:
            df2.loc[i, "Biome"] = a.biome_name
            df2.loc[i, "Lineage"] = a.lineage
        for g in std.geocoordinates:
            df2.loc[i, "Latitude"] = g.latitude
            df2.loc[i, "Longitude"] = g.longitude
        for p in std.publications:
            df2.loc[i, "Publication"] = p.doi

100%|██████████| 12/12 [00:44<00:00,  3.71s/it]


In [41]:
#Merge & Filter Table
df3 = pd.concat([df_study, df2], axis=1)
df3.set_index(["secondary_accession"])
df_study_biome = df3.query('Biome == ["root", "Algae"] and Lineage == ["root:Host-associated:Algae"]')
df_study_biome

,secondary_accession,url,samples_count,bioproject,accession,biomes,centre_name,last_update,study_abstract,is_public,study_name,data_origination,Biome,Lineage,Latitude,Longitude,Publication
10,ERP104738,https://www.ebi.ac.uk/metagenomics/api/v1/stud...,69,PRJEB23005,MGYS00002473,"[OrderedDict([('type', 'biomes'), ('id', 'root...",Alfred Wegener Institute Helmholtz Centre for ...,2018-06-29T14:31:43,We aimed to explore the community composition ...,True,Microbial diversity of the central Arctic Ocean,SUBMITTED,Algae,root:Host-associated:Algae,81.9255,131.129,10.3389/fmicb.2018.01035


In [17]:
#Export study data
df_study_biome.to_csv('02_Study_'+term+'+'+biome+'+'+lineage+'_filtered.csv')

In [21]:
#Selected study
from mg_toolkit.metadata import OriginalMetadata
om = OriginalMetadata("ERP104738")
om.save_to_csv(om.fetch_metadata(),"ERP104738_metadata.csv")


In [25]:
#Selected study
study = 'ERP104738'

In [43]:
#Fetch a list of sample data from a given study
filename_sample = '03_sample_'+study+'_raw.csv'
print(filename_sample)
if not os.path.isfile(filename_sample):
    with open(filename_sample, 'wb') as f:
        c = pycurl.Curl()
        c.setopt(c.URL, 'https://www.ebi.ac.uk/metagenomics/api/v1/studies/MGYS00002473/analyses?include=sample&format=csv')
        c.setopt(c.WRITEDATA, f)
        c.perform()
        c.close()

03_sample_ERP104738_raw.csv


In [44]:
#Filter relevant information from the list
df_sample = pd.read_csv(filename_sample)
df_sample_refine = df_sample[["accession","run","pipeline_version"]]

In [51]:
#check available metadata key
with Session(API_BASE) as s:
    s_meta = s.get('samples', df_sample_refine.loc[0, "accession"]).resource
s_meta.sample_metadata

DocumentError: Error 404: ?

In [50]:
#Create Container
if not os.path.isfile('04_raw_meta_'+study+'.csv'):
    df_meta = DataFrame(columns=('environmental package',"total depth of water column", "host-associated environmental package"))
    df_meta.index.name = 'No'
else:
    df_meta = pd.read_csv('04_raw_meta_'+study+'.csv', index_col=0)